In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

In [2]:
class SpotifyRecommender:
    def __init__(self, n_neighbors=11, metric='cosine'):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.model = NearestNeighbors(n_neighbors=n_neighbors, metric=metric, algorithm='brute')
        self.scaler = MinMaxScaler()
        # Genre'ları one-hot yapmak için memory efficient bir yöntem kullanacağız
        self.feature_matrix = None
        self.df = None
        
    def preprocess(self, df):
        """
        Ham veriyi alır, temizler ve modele girecek matrisi oluşturur.
        """
        print(">> Ön işleme başladı...")
        self.df = df.copy()
        
        # 1. Temizlik
        self.df = self.df.drop_duplicates(subset=['track_id']).reset_index(drop=True)
        self.df = self.df.dropna(subset=['track_name']) # İsimsiz şarkıları at
        
        # 2. Feature Selection
        numeric_cols = ['danceability', 'energy', 'loudness', 'speechiness', 
                        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
        
        # Kolon kontrolü
        available_cols = [c for c in numeric_cols if c in self.df.columns]
        
        # 3. Scaling (Sayısal Veriler)
        X_numeric = self.scaler.fit_transform(self.df[available_cols])
        
        # 4. Encoding (Kategorik Veriler - Genre)
        # Genre sütunu varsa One-Hot Encoding yap, yoksa sadece numeriği kullan
        genre_col = 'track_genre' if 'track_genre' in self.df.columns else 'genre'
        
        if genre_col in self.df.columns:
            # Pandas get_dummies kullanarak hızlıca one-hot yapalım
            # (Gerçek production ortamında sklearn OneHotEncoder daha güvenlidir ama bootcamp için bu yeterli)
            genre_dummies = pd.get_dummies(self.df[genre_col]).values
            
            # Matrisleri birleştir: [Ses Özellikleri | Tür Bilgisi]
            # Tür bilgisinin ağırlığını dengelemek için 0.5 ile çarpabiliriz (Opsiyonel Optimization)
            self.feature_matrix = np.hstack([X_numeric, genre_dummies * 0.5])
        else:
            self.feature_matrix = X_numeric
            
        print(f">> Ön işleme tamamlandı. Matris Boyutu: {self.feature_matrix.shape}")
        
    def fit(self):
        """
        Modeli oluşturulan matris ile eğitir.
        """
        print(">> Model eğitimi başladı...")
        self.model.fit(self.feature_matrix)
        print(">> Model eğitildi.")
        
    def recommend(self, song_name, n_recommendations=10):
        """
        Şarkı ismine göre öneri yapar.
        """
        # Şarkıyı bul (Case insensitive)
        mask = self.df['track_name'].str.contains(song_name, case=False, na=False)
        
        if not mask.any():
            return f"Hata: '{song_name}' bulunamadı."
        
        # İlk eşleşen şarkının indeksini al
        idx = self.df[mask].index[0]
        artist = self.df.loc[idx, 'artists']
        print(f"Referans: {self.df.loc[idx, 'track_name']} - {artist}")
        
        # Tahmin
        distances, indices = self.model.kneighbors(self.feature_matrix[idx].reshape(1, -1))
        
        # Sonuçları hazırla
        results = []
        for i, neighbor_idx in enumerate(indices[0][1:]): # Kendisini atla
            song = self.df.iloc[neighbor_idx]
            results.append({
                'Sıra': i+1,
                'Şarkı': song['track_name'],
                'Sanatçı': song['artists'],
                'Mesafe': distances[0][i+1]
            })
            
        return pd.DataFrame(results)

    def save_pipeline(self, path='../models/spotify_pipeline.pkl'):
        """
        Tüm sınıfı (veri + model + scaler) tek dosya olarak kaydeder.
        """
        joblib.dump(self, path)
        print(f">> Pipeline kaydedildi: {path}")

print("SpotifyRecommender sınıfı tanımlandı.")

SpotifyRecommender sınıfı tanımlandı.


In [3]:
# 1. Veriyi Yükle
raw_df = pd.read_csv('../data/Spotify_dataset.csv')

In [5]:
# 2. Pipeline Başlat
recommender = SpotifyRecommender(n_neighbors=11, metric='cosine')

In [6]:
# 3. Veriyi İşle (Preprocess)
recommender.preprocess(raw_df)

>> Ön işleme başladı...
>> Ön işleme tamamlandı. Matris Boyutu: (89740, 122)


In [7]:
# 4. Modeli Eğit (Train)
recommender.fit()

>> Model eğitimi başladı...
>> Model eğitildi.


In [8]:
# 5. Pipeline'ı Kaydet (Deploy için gerekli)
recommender.save_pipeline('../models/recommender_pipeline.pkl')

>> Pipeline kaydedildi: ../models/recommender_pipeline.pkl


In [9]:
# Örnek Test
print("\n--- TEST SONUCU ---")
test_results = recommender.recommend("Mockingbird") # Eminem
print(test_results)


--- TEST SONUCU ---
Referans: Listen To The Mockingbird - The Kentucky Colonels
   Sıra                                             Şarkı  \
0     1                                      Pig In A Pen   
1     2                               Brown's Ferry Blues   
2     3                              Uncle Harvey's Plane   
3     4                                Good Hearted Woman   
4     5  The Fox - Live From The Freight And Salvage/2000   
5     6                  You Were On My Mind This Morning   
6     7                                    Busted in Utah   
7     8                                          Jug Band   
8     9                               Old Man at the Mill   
9    10                                  I Know You Rider   

                       Sanatçı    Mesafe  
0             Old & In The Way  0.006609  
1      Doc Watson;Merle Watson  0.006727  
2        The Devil Makes Three  0.006763  
3  Yonder Mountain String Band  0.007129  
4                 Nickel Creek  

## Final Pipeline Raporu

### Pipeline Mimarisi
Proje, `SpotifyRecommender` isimli bir Python sınıfı (Class) üzerine inşa edilmiştir. Bu yapı şu adımları otomatikleştirir:
1.  **Preprocessing:** Veri temizliği ve nümerik değerlerin 0-1 arasına çekilmesi (MinMaxScaler).
2.  **Feature Construction:** Ses özellikleri ile müzik türü (Genre - OneHotEncoding) birleştirilerek hibrit bir matris oluşturulmuştur.
3.  **Training:** Optimize edilen parametrelerle (Metric: Cosine, K: 11) KNN modeli eğitilmiştir.

### Karar Gerekçeleri
* **Neden KNN?** 
    * Dataset boyutu (114k) KNN için yönetilebilir seviyededir.
    * Müzik önerisinde "benzerlik" kavramı en iyi vektör uzayındaki mesafelerle açıklanabilir.
* **Neden Class Yapısı?**
    * Modeli bir API veya Streamlit uygulamasına taşırken kod tekrarını önlemek için tek bir obje (Object Oriented) yapısı tercih edilmiştir.